In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [ ]:
ls

longlat.csv  sample_data/  spark-3.1.1-bin-hadoop2.7.tgz


In [ ]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 16:46:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.236.206.131, 3.229.173.44, 3.225.89.236, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.236.206.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  18.5MB/s    in 0.8s    

2021-03-31 16:46:41 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  tes

In [ ]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv ##Data set download

In [ ]:
mv rows.csv col.csv ## Linux cmd to rename.

In [ ]:
ls

col.csv      ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7/
longlat.csv  sample_data/                  spark-3.1.1-bin-hadoop2.7.tgz


In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [ ]:
spark =SparkSession(sc)

In [ ]:

# Reading data from local repo. to pyspark.

col=spark.read.option("header",True).csv("./col.csv",inferSchema=True) # data set 1.
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True) #location. [Additional Dataset ]

In [ ]:
col.count() - col.distinct().count() # I checked for Duplicate rows in our dataset.

0

In [ ]:
len(col.columns) # how many columns do I have.

29

In [ ]:
col.printSchema() #Schema of my dataset. Issues with our columns types like NUMBER OF PERSONS INJURED: string and It should need to be Integer.

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [ ]:
from pyspark.sql.types import IntegerType

col=col.withColumn("NUMBER OF PERSONS INJURED", col["NUMBER OF PERSONS INJURED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF CYCLIST KILLED", col["NUMBER OF CYCLIST KILLED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF MOTORIST INJURED", col["NUMBER OF MOTORIST INJURED"].cast(IntegerType()))

# Correcting the column types to Integer which were previously of String types.

In [ ]:
col.printSchema() # checking if my schema is correct now ? Yes It is

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: st

In [ ]:
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|05/06/2013|
|05/08/2013|
|05/06/2013|
|05/07/2013|
|05/07/2013|
|05/09/2013|
|05/08/2013|
|05/09/2013|
|05/05/2013|
|05/08/2013|
+----------+
only showing top 10 rows



In [ ]:
#dropping columns with collision id = null

In [ ]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: int, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: int, NUMBER OF MOTORIST INJURED: int, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [ ]:
col.count()

1766160

In [ ]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [ ]:
cannotRetriveLocation = missingLocation.filter(col["LONGITUDE"].isNull())

In [ ]:
col.count() ## removed the records for which we cannot retrive location at any cost.

1766160

In [ ]:
#Infer the schema, and register the DataFrame as a table.
col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [ ]:
col=spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [ ]:
col.count()

1588740

In [ ]:
col.filter(col["ZIP CODE"].isNull()).filter(col["BOROUGH"].isNotNull()).count()

211

In [ ]:
BOROUGH=list(map(lambda x : x["BOROUGH"],col.select("BOROUGH").distinct().collect()))

In [ ]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [ ]:
longlat.printSchema() # another data set

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [ ]:

col.createOrReplaceTempView("col");
longlat.createOrReplaceTempView("longlat");

In [ ]:
missingZipCode=col.filter(col['ZIP CODE'].isNull());

In [ ]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [ ]:
"""""""""""
for m in missingZipCode.limit(19).collect():
  zip=spark.sql("select zip,city FROM longlat where ROUND(longlat.latitude,2) ="+str(round(m.LATITUDE,2))+" and ROUND(longlat.longitude,2)= "+str(round(m.LONGITUDE,2))+" limit 1");
  zip=zip.select("zip","city").collect()
  if len(zip)>0:
    if zip[0].city.upper()=="NEW YORK":
      print(m["COLLISION_ID"],"MANHATTAN",zip[0].zip)
    else:
        if zip[0].city.upper() in BOROUGH:
          print(m["COLLISION_ID"],zip[0].city.upper(),zip[0].zip)
          """"""

SyntaxError: ignored

In [ ]:
col.select("NUMBER OF CYCLIST INJURED").distinct().show(); ## lets check.

NUMBER OF CYCLIST INJURED Is alredy cleaned.

In [ ]:
col.select("NUMBER OF PERSONS INJURED").distinct().show();

NUMBER OF PERSONS INJURED has Null value hence making it to Zero.

In [ ]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS INJURED"]) # clean null values to zero.

In [ ]:
#NUMBER OF PERSONS KILLED
print(col.select("NUMBER OF PERSONS KILLED").distinct().show());

NUMBER OF PERSONS KILLED has NULL values : replacign them with 0.


In [ ]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS KILLED"])

In [ ]:
print(col.select("NUMBER OF PEDESTRIANS INJURED").distinct().show());

NUMBER OF PEDESTRIANS INJURED has no null value

In [ ]:
#NUMBER OF PEDESTRIANS KILLED
print(col.select("NUMBER OF PEDESTRIANS KILLED").distinct().show());



```
NUMBER OF PEDESTRIANS KILLED has no null value
```



In [ ]:
print(col.select("NUMBER OF CYCLIST INJURED").distinct().show());

NUMBER OF CYCLIST INJURED has no null value

In [ ]:
print(col.select("NUMBER OF CYCLIST KILLED").distinct().show());

NUMBER OF CYCLIST KILLED has no null value

In [ ]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST INJURED").distinct().show(100));

NUMBER OF MOTORIST INJURED has no null value

In [ ]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST KILLED").distinct().show(100));

NUMBER OF MOTORIST INJURED has no null value

In [ ]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 1").distinct().show(70));

# has numerical values 1 and 80
# has null value.


In [ ]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 2").distinct().show(100));

# has numerical values 1 and 80.
# has null value.


In [ ]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 3").distinct().show(100));

# has numerical values 1 and 80.
# has null value.

In [ ]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 4").distinct().show(100));
# has null value.

In [ ]:
print(col.select("CONTRIBUTING FACTOR VEHICLE 5").distinct().show(30));
# already Clean

In [ ]:
print(col.select("VEHICLE TYPE CODE 1").distinct().show(1000));

In [ ]:
col.select("VEHICLE TYPE CODE 1").distinct().count()

In [ ]:
col.select("VEHICLE TYPE CODE 2").distinct().count()

In [ ]:
col.select("VEHICLE TYPE CODE 3").distinct().count()

In [ ]:
col.select("VEHICLE TYPE CODE 4").distinct().count()

# ***TODO***
Treat Null Values for all column to  Unknown. 1,2,3,4,5


Treat **1 and 80** for CONTRIBUTING FACTOR VEHICLE 1,2,3

In [ ]:
# zip,borough,location,long,lat, [contributing1,2,3,4], killed and injured [3-4],COLLISION_ID.

# Data cleaning for Person table

In [ ]:
!wget -q https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv   #Loaded person data for vehicle collisions

^C


In [ ]:
mv rows.csv person.csv

In [ ]:
ls

col.csv                       sample_data/
longlat.csv                   spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7.tgz
person.csv


In [ ]:
person = spark.read.option("header",True).csv("./person.csv",inferSchema=True) # Read person data.

In [ ]:
person.createOrReplaceTempView("person");

In [ ]:
person.count()

4236940

In [ ]:
len(person.columns)

21

In [ ]:
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = true)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = true)
 |-- EMOTIONAL_STATUS: string (nullable = true)
 |-- BODILY_INJURY: string (nullable = true)
 |-- POSITION_IN_VEHICLE: string (nullable = true)
 |-- SAFETY_EQUIPMENT: string (nullable = true)
 |-- PED_LOCATION: string (nullable = true)
 |-- PED_ACTION: string (nullable = true)
 |-- COMPLAINT: string (nullable = true)
 |-- PED_ROLE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)
 |-- PERSON_SEX: string (nullable = true)



In [ ]:
person.count() - person.distinct().count()    #Check for duplicates

0

In [ ]:
person.filter(person.COLLISION_ID.isNull()).drop()    #Check and remove Null values for Foreign key

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [ ]:
person.filter(person.UNIQUE_ID.isNull()).drop()     #Check and remove Null values for Primary key

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [ ]:
person.filter(person.PERSON_ID.isNull()).count()    

19

In [ ]:
person=person.fillna(value='Unknown',subset=["PERSON_ID"])

In [ ]:
print(person.select("PERSON_TYPE").distinct().show(100))  # Cleaned

+-----------+
|PERSON_TYPE|
+-----------+
|   Occupant|
| Pedestrian|
|  Bicyclist|
+-----------+

None


In [ ]:
print(person.select("PERSON_INJURY").distinct().show(100)) #Already Clean

+-------------+
|PERSON_INJURY|
+-------------+
|       Killed|
|  Unspecified|
|      Injured|
+-------------+

None


In [ ]:
person.select("VEHICLE_ID").distinct().count()

1939149

In [ ]:
person.filter(person.VEHICLE_ID.isNull()).count()

165791

In [ ]:
person=person.fillna(value='-999',subset=["VEHICLE_ID"])

In [ ]:
person.filter(person.PERSON_AGE.isNull()).count()

363520

In [ ]:
mean = spark.sql("Select sum(PERSON_AGE)/count(PERSON_AGE) from person where PERSON_AGE > 0 and PERSON_AGE < 100");

In [ ]:
mean.show()

+-------------------------------------------------------------------------------------+
|(CAST(sum(CAST(PERSON_AGE AS BIGINT)) AS DOUBLE) / CAST(count(PERSON_AGE) AS DOUBLE))|
+-------------------------------------------------------------------------------------+
|                                                                    40.24500953358535|
+-------------------------------------------------------------------------------------+



In [ ]:
mean.createOrReplaceTempView("mean")

In [ ]:
mean = spark.sql("SELECT cast((select * from mean) as int)")

In [ ]:
from pyspark.sql.functions import when

In [ ]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] > 100, 40).otherwise(person["PERSON_AGE"]))

In [ ]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] < 0, 40).otherwise(person["PERSON_AGE"]))

In [ ]:
person=person.fillna(value=40,subset=["PERSON_AGE"])

In [ ]:
person.select("PERSON_AGE").distinct().show()

+----------+
|PERSON_AGE|
+----------+
|        31|
|        85|
|        65|
|        53|
|        78|
|        34|
|        81|
|        28|
|        76|
|        27|
|        26|
|        44|
|        12|
|        91|
|        22|
|        93|
|        47|
|         1|
|        52|
|        13|
+----------+
only showing top 20 rows



In [ ]:
person.select("EJECTION").distinct().show()

+-----------------+
|         EJECTION|
+-----------------+
|Partially Ejected|
|   Does Not Apply|
|             null|
|          Unknown|
|      Not Ejected|
|          Ejected|
|          Trapped|
+-----------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["EJECTION"])  #Cleaned EJECTED

In [ ]:
person.select("EMOTIONAL_STATUS").distinct().show()

+----------------+
|EMOTIONAL_STATUS|
+----------------+
|      Incoherent|
|  Does Not Apply|
|   Semiconscious|
|            null|
|         Unknown|
|     Unconscious|
|           Shock|
|  Apparent Death|
|       Conscious|
+----------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["EMOTIONAL_STATUS"])  #Cleaned EMOTIONAL_STATUS

In [ ]:
person.select("BODILY_INJURY").distinct().show()

+--------------------+
|       BODILY_INJURY|
+--------------------+
|      Does Not Apply|
|                null|
|                Head|
|             Unknown|
|                Neck|
|Elbow-Lower-Arm-Hand|
|         Entire Body|
|                Face|
|                Back|
|       Hip-Upper Leg|
|    Abdomen - Pelvis|
|                 Eye|
|Shoulder - Upper Arm|
|               Chest|
| Knee-Lower Leg Foot|
+--------------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["BODILY_INJURY"])  #Cleaned BODILY_INJURY

In [ ]:
person.select("PERSON_SEX").distinct().show()

+----------+
|PERSON_SEX|
+----------+
|         F|
|      null|
|         M|
|         U|
+----------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["PERSON_SEX"])  #Cleaned PERSON_SEX

In [ ]:
person.select("POSITION_IN_VEHICLE").distinct().show()

+--------------------+
| POSITION_IN_VEHICLE|
+--------------------+
|Riding/Hanging on...|
|Front passenger, ...|
|Any person in the...|
|If one person is ...|
|      Does Not Apply|
|              Driver|
|Right rear passen...|
|                null|
|Middle front seat...|
|             Unknown|
|Left rear passeng...|
|Middle rear seat,...|
+--------------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["POSITION_IN_VEHICLE"])

In [ ]:
person.select("SAFETY_EQUIPMENT").distinct().show()

+--------------------+
|    SAFETY_EQUIPMENT|
+--------------------+
|                None|
|Stoppers Only (In...|
|Pads Only (In-Lin...|
|                null|
|    Air Bag Deployed|
|Air Bag Deployed/...|
|Air Bag Deployed/...|
|             Unknown|
|               Other|
|                   -|
|             Harness|
|Helmet/Other (In-...|
|Child Restraint Only|
|    Lap Belt/Harness|
|Helmet (Motorcycl...|
|  Lap Belt & Harness|
|Helmet Only (In-L...|
|Air Bag Deployed/...|
|            Lap Belt|
+--------------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["SAFETY_EQUIPMENT"])

In [ ]:
person.select("PED_LOCATION").distinct().show()

+--------------------+
|        PED_LOCATION|
+--------------------+
|      Does Not Apply|
|                null|
|             Unknown|
|Pedestrian/Bicycl...|
|Pedestrian/Bicycl...|
+--------------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["PED_LOCATION"])

In [ ]:
person.select("PED_ACTION").distinct().show()

+--------------------+
|          PED_ACTION|
+--------------------+
|Pushing/Working o...|
|      Not in Roadway|
|Riding/Walking Al...|
|Crossing, No Sign...|
|      Does Not Apply|
|                null|
|Other Actions in ...|
|             Unknown|
|  Working in Roadway|
|Getting On/Off Ve...|
|Emerging from in ...|
|  Playing in Roadway|
|Crossing Against ...|
|Crossing, No Sign...|
|Going to/From Sto...|
|Crossing With Signal|
|Riding/Walking Al...|
+--------------------+



In [ ]:
person=person.fillna(value='Unspecified',subset=["PED_ACTION"])

In [ ]:
person.select("COMPLAINT").distinct().show()

+--------------------+
|           COMPLAINT|
+--------------------+
|   Complaint of Pain|
|          Minor Burn|
|  Contusion - Bruise|
|      Does Not Apply|
|         Severe Burn|
|                null|
|      Minor Bleeding|
|      Crush Injuries|
|           Paralysis|
|             Unknown|
|            Abrasion|
|Fracture - Disloc...|
|        None Visible|
|            Internal|
|            Whiplash|
|Fracture - Distor...|
|          Concussion|
|     Severe Bleeding|
|  Severe Lacerations|
|Complaint of Pain...|
+--------------------+
only showing top 20 rows



In [ ]:
person=person.fillna(value='Unspecified',subset=["COMPLAINT"])

In [ ]:
person.select("PED_ROLE").distinct().show()

+---------------+
|       PED_ROLE|
+---------------+
|        Witness|
|         Driver|
|           null|
|Notified Person|
|  Policy Holder|
|          Other|
|      Passenger|
|     Registrant|
|     Pedestrian|
|          Owner|
| In-Line Skater|
+---------------+



In [ ]:
person=person.fillna(value='Unknown',subset=["PED_ROLE"])

In [ ]:
person.select("CONTRIBUTING_FACTOR_1").distinct().show(100)

+---------------------+
|CONTRIBUTING_FACTOR_1|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Windshield Inadeq...|
| Using On Board Na...|
|         Unsafe Speed|
| Shoulders Defecti...|
|       Tinted Windows|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
| Other Lighting De...|
|                 null|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Contro

In [ ]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_1"])

In [ ]:
person.select("CONTRIBUTING_FACTOR_2").distinct().show(100)

+---------------------+
|CONTRIBUTING_FACTOR_2|
+---------------------+
| Following Too Clo...|
| Traffic Control D...|
| Driverless/Runawa...|
| Accelerator Defec...|
|         Unsafe Speed|
|    Oversized Vehicle|
| Passing or Lane U...|
| Lane Marking Impr...|
|                 null|
| Aggressive Drivin...|
|      Other Vehicular|
|  Driver Inexperience|
|              Texting|
|               Illnes|
|      Drugs (illegal)|
|          Unspecified|
|   Pavement Defective|
| Prescription Medi...|
| View Obstructed/L...|
|   Lost Consciousness|
| Reaction to Uninv...|
|          Fell Asleep|
| Tire Failure/Inad...|
| Outside Car Distr...|
|      Fatigued/Drowsy|
| Driver Inattentio...|
| Cell Phone (hand-...|
| Listening/Using H...|
| Failure to Keep R...|
|   Obstruction/Debris|
| Unsafe Lane Changing|
| Pedestrian/Bicycl...|
| Passenger Distrac...|
|  Alcohol Involvement|
| Traffic Control D...|
|     Backing Unsafely|
|     Brakes Defective|
|   Eating or Drinking|
| Failure to Yie

In [ ]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_2"])

In [ ]:
#Change CRASH DATE from String to date
from pyspark.sql.types import DataType
from pyspark.sql.functions import to_date

person = person.withColumn('CRASH_DATE', to_date(person["CRASH_DATE"], 'MM/dd/yyyy'))
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = true)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = true)
 |-- EMOTIONAL_STATUS: string (nullable = true)
 |-- BODILY_INJURY: string (nullable = true)
 |-- POSITION_IN_VEHICLE: string (nullable = true)
 |-- SAFETY_EQUIPMENT: string (nullable = true)
 |-- PED_LOCATION: string (nullable = true)
 |-- PED_ACTION: string (nullable = true)
 |-- COMPLAINT: string (nullable = true)
 |-- PED_ROLE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)
 |-- PERSON_SEX: string (nullable = true)



In [ ]:
from pyspark.sql.functions import to_timestamp
person = person.withColumn('CRASH_TIME', to_timestamp(person["CRASH_TIME"], 'H:mm'))
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: timestamp (nullable = true)
 |-- PERSON_ID: string (nullable = true)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = true)
 |-- EMOTIONAL_STATUS: string (nullable = true)
 |-- BODILY_INJURY: string (nullable = true)
 |-- POSITION_IN_VEHICLE: string (nullable = true)
 |-- SAFETY_EQUIPMENT: string (nullable = true)
 |-- PED_LOCATION: string (nullable = true)
 |-- PED_ACTION: string (nullable = true)
 |-- COMPLAINT: string (nullable = true)
 |-- PED_ROLE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)
 |-- PERSON_SEX: string (nullable = true)



In [ ]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f
person = person.withColumn('CRASH_TIME', f.date_format("CRASH_TIME", 'HH:mm'))

In [ ]:
person.select("CRASH_TIME").show(10)

+----------+
|CRASH_TIME|
+----------+
|     09:43|
|     15:15|
|     17:55|
|     13:05|
|     11:16|
|     19:15|
|     16:45|
|     01:15|
|     13:04|
|     08:20|
+----------+
only showing top 10 rows



In [ ]:
person.select("CRASH_DATE").show(10)

+----------+
|CRASH_DATE|
+----------+
|2019-10-26|
|2019-10-25|
|2019-10-26|
|2016-11-21|
|2019-10-25|
|2019-10-24|
|2019-10-26|
|2019-10-26|
|2019-10-26|
|2019-10-24|
+----------+
only showing top 10 rows

